In [1]:
# import common libraries
%load_ext autoreload
%autoreload 2

%matplotlib notebook

In [2]:
import os

import numpy as np
import pandas as pd

import lightgbm

# Our Proposal

In [10]:
dataset_file = "census.csv"

In [11]:

# load

dataset_df = pd.read_csv(dataset_file, sep=",")

# replace -1 with 0 in the label
# dataset_df['income_greater_than_50k'][ dataset_df['income_greater_than_50k'] == -1  ] = 0

# simulate attacks of len 10
attack_lens = [ 10 for i in range(len(dataset_df)//10) ]
attack_lens += [len(dataset_df)%10] 

attack_lens = np.ones(len(dataset_df),dtype=np.int ) 

train_lgb = lightgbm.Dataset( data  = dataset_df.iloc[:,:-1].values, 
                              label = dataset_df.iloc[:,-1].values, 
                              group = attack_lens)



In [6]:
# our true metric

# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool

def attack_eval(preds, train_data):
    labels      = train_data.get_label()
    attack_lens = train_data.get_group()
    
    norm = 1.0 / float(len(labels))
        
    exp_loss = lambda h,t: np.log( 1.0 + np.exp(-t*h) )
    
    sum_max = 0.0
    offset = 0
    for atk in attack_lens:
        losses = [ exp_loss(h,t) for h,t in zip(preds[offset:offset+atk], labels[offset:offset+atk]) ]
        sum_max += max(losses)
        
        offset += atk
    
    avg_loss = norm * sum_max
    
    return 'X-adv', avg_loss, False

In [13]:
# self-defined objective function
# f(preds: array, train_data: Dataset) -> grad: array, hess: array

def attack_grad(preds, train_data):
    labels      = train_data.get_label()
    attack_lens = train_data.get_group()
    e_preds = preds
    
    grads = np.zeros_like(labels, dtype=np.float64)
    hess  = np.zeros_like(grads)

    norm = 1.0 / float(len(labels))
    
    offset = 0
    for atk in attack_lens:
        inv_sum = 1.0 / np.sum( 1.0 + np.exp(- e_preds[offset:offset+atk] * labels[offset:offset+atk]) )

        for x in range(atk):
            x_loss = np.exp( - labels[offset+x] * e_preds[offset+x] ) 
            
            x_grad = inv_sum * x_loss
            
#            print (norm * x_grad * (- labels[offset+x]))
            
            grads[offset+x] = norm * x_grad * (- labels[offset+x])
#            print (x_grad, norm)
            
            hess[offset+x]  = norm * x_grad * (1.0 - x_grad)
            
        offset += atk
        
    return grads, hess

In [14]:
params = {
    'learning_rate': 0.1,
    'num_leaves': 32,
    'min_data_in_leaf': 5
}    

lgbm_info = {}

lgbm_model = lightgbm.train( params, train_lgb, num_boost_round=10,
                             feval = attack_eval,
                             fobj  = attack_grad,
                             valid_sets = [train_lgb], 
                             evals_result = lgbm_info,
                             verbose_eval= True )



lgbm_info



[1]	training's X-adv: 0.637024
[2]	training's X-adv: 0.591143
[3]	training's X-adv: 0.552535
[4]	training's X-adv: 0.519896
[5]	training's X-adv: 0.49259
[6]	training's X-adv: 0.469267
[7]	training's X-adv: 0.449134
[8]	training's X-adv: 0.431365
[9]	training's X-adv: 0.415989
[10]	training's X-adv: 0.402811


{'training': defaultdict(list,
             {'X-adv': [0.6370235022114553,
               0.5911428919150238,
               0.5525352394904304,
               0.519895506290343,
               0.4925900377997718,
               0.46926737486227543,
               0.4491338717819763,
               0.4313651191801964,
               0.4159888290017059,
               0.4028105177822682]})}

# Adversarial Training a.k.a. fast gradient sign method

Paper: "EXPLAINING AND HARNESSING ADVERSARIAL EXAMPLES", Ian J. Goodfellow, Jonathon Shlens & Christian Szegedy. ICLR 2015.


Method cannot be applied directly.

We assume to have in input the attacked datasets, where the first instance of each group is the original one, and the other are the attacks.

The method selects just one attack, i.e., the one somehow close to the greatest gradient, i.e., the one causing the greatest loss.

The resulting dataset has the same number of original and attacked instances.

The process is repeated at each iteration/tree.

In [3]:
# this almost a copy paste from the eval function, not very nice
def pointwise_loss(raw_data, raw_labels, raw_groups, model):
    
    # compute preds on the full attack dataset
    preds = model.predict(raw_data)

    # logloss
    log_loss = lambda h,t: np.log( 1.0 + np.exp(-t*h) )
    
    # compute loss on every instance
    losses = np.array( [ log_loss(h,t) for h,t in zip(preds, raw_labels) ] )

    # nomalize (useless)
    norm = 1.0 / float(len(raw_labels))
    losses = norm * losses
    
    return losses

# crate the (LGBM) adversarial dataset 
def adv_training(raw_data, raw_labels, raw_groups, model):
    
    if model==None:
        # first iteration, get only original instances
        orig_instances = [0] + list( np.cumsum(raw_groups[:-1]) )
        new_rows   = raw_data[orig_instances,:]
        new_labels = raw_labels[orig_instances]
        new_groups = np.ones_like(raw_groups, dtype=np.int)

    else:
        # compute pointwise losses
        losses = pointwise_loss(raw_data, raw_labels, raw_groups, model)
        # new instances to be added to the model
        new_instances = []
        new_groups = []
        # iterate groups
        offset = 0
        for atk in raw_groups:
            new_instances += [offset] # keep original
            new_groups    += [1]      # one instance in the group
            if atk>1:
                # get instance with max loss (not including the first in the group)
                adv_instance  = np.argmax(losses[offset+1:offset+atk])
                new_instances += [adv_instance + 1]
                new_groups[-1] = 2    # two instances in the group
                
            offset += atk

        # select relevant rows
        new_instances = np.array(new_instances, dtype=np.int) # maybe this speeds up a bit
        new_rows   = raw_data[new_instances,:]
        new_labels = raw_labels[new_instances]
        new_groups = np.array(new_groups, dtype=np.int)

    # create the training dataset for LGBM
    new_training = lightgbm.Dataset( data  = new_rows, label = new_labels, group = new_groups)
    
    return new_training

# self-defined objective function
# f(preds: array, train_data: Dataset) -> grad: array, hess: array

def fgsm_grad(preds, train_data):
    labels      = train_data.get_label()
    
    # compute derivative of logistic loss
    exp_th = np.exp(preds * labels)
    grads = -labels / (1.0 + exp_th) 
    hess  = exp_th * (1.0 / (1.0 + exp_th)**2 )
    
    # normalize
    norm = 1.0 / float(len(labels))
    grads *= norm
    hess *= norm
        
    return grads, hess

In [4]:

# load
# !!!! This should load the attacked dataset and not the original one
dataset_file = "census.csv"
dataset_df = pd.read_csv(dataset_file, sep=",")

# simulate attacks of len 10
attack_lens = [ 10 for i in range(len(dataset_df)//10) ]
attack_lens += [len(dataset_df)%10] 

# attack_lens = np.ones(len(dataset_df),dtype=np.int ) 

In [7]:

NUM_TREES = 10

lgbm_model = None
for t in range(NUM_TREES):
    print ("Tree:", 1)
    
    # create adversarial dataset
    
    # dataset_df is something coming from gabriele
    # you can avoid dataframes and use original data
    adv_train = adv_training( dataset_df.iloc[:,:-1].values, 
                              dataset_df.iloc[:,-1].values, 
                              attack_lens, 
                              lgbm_model)
    
    
    # training params
    params = {
        'learning_rate': 0.1,
        'num_leaves': 32,
        'min_data_in_leaf': 5
    }    

    # training eval stats
    lgbm_info = {}

    # add a new tree to the previous model 
    #   by using the new adversarial training dataset
    lgbm_model = lightgbm.train( params, adv_train, 
                                 init_model = lgbm_model,
                                 num_boost_round=1,
                                 feval = attack_eval,
                                 fobj  = fgsm_grad,
                                 valid_sets = [adv_train], 
                                 evals_result = lgbm_info,
                                 verbose_eval = True )

    

Tree: 1
[1]	training's X-adv: 0.635451
Tree: 1
[2]	training's X-adv: 0.306789
Tree: 1
[3]	training's X-adv: 0.284487
Tree: 1
[4]	training's X-adv: 0.266459
Tree: 1
[5]	training's X-adv: 0.251478
Tree: 1
[6]	training's X-adv: 0.239128
Tree: 1
[7]	training's X-adv: 0.228335
Tree: 1
[8]	training's X-adv: 0.218966
Tree: 1
[9]	training's X-adv: 0.210484
Tree: 1
[10]	training's X-adv: 0.203279
